<img src="Figures/top_ML.png" alt="Drawing" style="width: 1000px;"/>

# "Digitalización de una carga doméstica"

## Postítulo "Digital Energy", Máster Smart Energy

## Módulo: Digitalización de la Energía

## Alumno: Max Antonio Missene Montecinos

### Objetivo:

#### Monitoreo de una carga doméstica mediante un procesador de datos construido en base a un microcontrolador Arduino.

### Pautas generales:

1. Descripción de la carga seleccionada para nuestro Proyecto. Modelo, potencia nominal, uso habitual que hacéis de ella, etc.
2. Descripción de vuestro montaje: Fotos del montaje con la carga, explicación breve del código para la realización de la medición.
3. Análisis preliminar de consumo de la carga: Captura del a corriente instantánea, grabar datos de corriente en RMS y realizar alguna gráfica (Excel mismo) y comentar el consumo observado.

## Generales

#### El primer paso que se realizará en el desarrollo del presente trabajo, será importar las librerías y funciones de librerías que utilizaremos a lo largo del código.

In [23]:
# Librerías usadas

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
from datetime import date, time, datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

#### A continuación, se procederá a importar la base de datos.

In [24]:
# Generar DataFrame (DF) a partir de base de datos de excel
columns=['Fecha','Corriente','Potencia']
BD_carga = pd.read_csv('Data/data_400_C_20h.csv', names=columns)

### 1. Entendimiento de los datos, estadística descriptiva y visualizaciones

#### En primera instancia, se determinará el número de dimensiones que posee la base de datos.

In [25]:
# Se procederá a conocer las dimensiones del DF con la función "shape".
BD_carga.shape

(122, 3)

#### Es posible notar que el DataFrame contine 3 columnas y 122 filas. 

#### Así, se desplegaran las primeras filas del dataframe, para examinar de forma preliminar los datos.

In [26]:
# Desplegar primeras 5 filas del DF a partir de la función "head".
BD_carga.head()

,Fecha,Corriente,Potencia
0,2022-06-07 20:20:33.270494,0.20,46.46
1,2022-06-07 20:20:34.274694,0.20,47.15
2,2022-06-07 20:20:35.277526,0.21,47.54
3,2022-06-07 20:20:36.276230,0.21,47.76
4,2022-06-07 20:20:37.280886,0.21,47.53


#### De manera análoga, se desplegaran las primeras últimas filas del dataframe, para examinar de forma preliminar los datos.

In [27]:
# Desplegar última 5 filas del DF a partir de la función "tail".
BD_carga.tail()

,Fecha,Corriente,Potencia
117,2022-06-07 20:22:30.529728,0.22,49.82
118,2022-06-07 20:22:31.528438,0.21,48.87
119,2022-06-07 20:22:32.531909,0.22,50.06
120,2022-06-07 20:22:33.535345,0.21,49.33
121,2022-06-07 20:22:34.535661,0.21,49.19


In [28]:
# Desplegar última 5 filas del DF a partir de la función "tail".
BD_carga.dtypes

Fecha         object
Corriente    float64
Potencia     float64
dtype: object

#### Ya pudiendo mirar el contenido del dataframe de manera general, se realizará el cálculo de valores estadísticos de sus columnas.

In [29]:
# Desplegar última 5 filas del DF a partir de la función "tail".
BD_carga['Fecha']=pd.to_datetime(BD_carga['Fecha'])
BD_carga.dtypes


Fecha        datetime64[ns]
Corriente           float64
Potencia            float64
dtype: object

In [30]:
# Se aplica la función "describe" al dataframe, el cual calculará algunos parámetros estadísticos a cada columna.
BD_carga.describe()

,Corriente,Potencia
count,122.000000,122.000000
mean,5.752541,1323.099508
std,1.844666,424.218825
min,0.200000,46.460000
25%,6.360000,1461.890000
50%,6.370000,1465.065000
75%,6.387500,1468.587500
max,6.560000,1508.730000


In [31]:
BD_carga.isna().sum()

Fecha        0
Corriente    0
Potencia     0
dtype: int64

In [33]:
BD_carga['Energy_kWh']=0.0
BD_carga['time_dif']=0.0
BD_carga['cum_energy_kwh']=0.0
longitud=len(BD_carga['Energy_kWh'])

for x in range(0, longitud):
 if BD_carga['Corriente'][x]<0.0:
        BD_carga.at[x, 'Corriente']=0.0
 if BD_carga['Potencia'][x]<0.0:
        BD_carga.at[x, 'Potencia']=0.0
for x in range(1, longitud):
    BD_carga.at[x, 'Energy_kWh']=BD_carga['Potencia'][x]*((BD_carga['Fecha'][x]-BD_carga['Fecha'][x-1]).total_seconds()/3600000)
    BD_carga.at[x, 'time_dif']=(BD_carga['Fecha'][x]-BD_carga['Fecha'][x-1]).total_seconds()
    BD_carga.at[x, 'cum_energy_kwh']=BD_carga['cum_energy_kwh'][x-1]+BD_carga['Energy_kWh'][x]
    

In [34]:
BD_carga.head()

,Fecha,Corriente,Potencia,Energy_kWh,time_dif,cum_energy_kwh
0,2022-06-07 20:20:33.270494,0.20,46.46,0.000000,0.000000,0.000000
1,2022-06-07 20:20:34.274694,0.20,47.15,0.000013,1.004200,0.000013
2,2022-06-07 20:20:35.277526,0.21,47.54,0.000013,1.002832,0.000026
3,2022-06-07 20:20:36.276230,0.21,47.76,0.000013,0.998704,0.000040
4,2022-06-07 20:20:37.280886,0.21,47.53,0.000013,1.004656,0.000053


In [35]:
BD_carga.tail()

,Fecha,Corriente,Potencia,Energy_kWh,time_dif,cum_energy_kwh
117,2022-06-07 20:22:30.529728,0.22,49.82,0.000014,1.004287,0.044870
118,2022-06-07 20:22:31.528438,0.21,48.87,0.000014,0.998710,0.044883
119,2022-06-07 20:22:32.531909,0.22,50.06,0.000014,1.003471,0.044897
120,2022-06-07 20:22:33.535345,0.21,49.33,0.000014,1.003436,0.044911
121,2022-06-07 20:22:34.535661,0.21,49.19,0.000014,1.000316,0.044925


In [36]:
BD_carga['Energy_kWh'].sum()

0.044924624185138894

In [37]:
((BD_carga['time_dif'].sum())/60)/60

0.03368476861111111

In [40]:
# conda install -c plotly plotly
import plotly.offline as py;
from plotly import tools;
import plotly.graph_objs as go;
from ipywidgets import widgets;
from ipywidgets import interact;
import warnings
warnings.filterwarnings('ignore')
py.init_notebook_mode(connected= True)

In [41]:
fig = go.Figure(
    data=[
        go.Line(x=BD_carga['Fecha'][0:12*75],
                y=BD_carga['Potencia'][0:12*75], name="Power")
    ],
    layout=go.Layout(
        title=dict(text='Potencia RMS Hervidor Eléctrico: 20h;AguaNormal;400ml'),
        xaxis=dict(title='Hora'),
        yaxis=dict(title='Potencia [W]')
    )
)
fig.show()
        
        